In [1]:
#remove irrelevant entries if the formulation was unspecified

def remove_irrelevant(input_array, phos_input, ion_input, peg_input, dna_input): 
    query_array = input_array

    #remove rows if the formulation doesn't use that component
    if phospholipid != 0:
        query_array = input_array[input_array[:,col_phos] == phospholipid]

    if ionizable_lipid != 0:
        query_array = query_array[query_array[:,col_ion] == ionizable_lipid]

    if lipid_PEG != 0:
        query_array = query_array[query_array[:,col_peg] == lipid_PEG]

    if surface_DNA != 0:
        query_array = query_array[query_array[:,col_dna] == surface_DNA]

    #Give a heads up
    if np.prod(query_array.shape) == 0:
        print("CAUTION: No training data for the combination of inputted reagents\n")
    
    return(query_array)
 


In [2]:
import sys
import numpy as np
import pandas as pd
from io import *
from sklearn import linear_model


#init lib path
allforms_path = "allforms_data_clean.csv"

#########################################################
###     init column vars      ###
#identities
col_lib = 0
col_phos = 1
col_ion = 2
col_peg = 3
col_dna = 4

#formulation (percent comp)
col_per_phos = 5
col_per_chol = 6
col_per_ion = 7
col_per_peg = 8

#results
col_encap = 9
col_size = 10 #in nm
col_stdev = 11 #nm

###########################################################

#init input vars. Users will be prompted to enter a number corresponded to the reagent provided in the prompt
phospholipid = 1
ionizable_lipid = 2
lipid_PEG = 2
surface_DNA = 0
user_size = 0

###########################################################

#convert raw lib into numpy array
allforms = np.genfromtxt(allforms_path, delimiter=",", skip_header=1)

#isolate relevant data
query_array = remove_irrelevant(allforms, phospholipid, ionizable_lipid, lipid_PEG, surface_DNA)





In [25]:

#define input/output arrays and build model

focus_percentage_training = query_array[:,(col_dna+1):(col_encap)] #molar percentages of relevant formulations
all_percentage_training = allforms[:,(col_dna+1):(col_encap)] #molar percentages of all formulations

focus_size_training = query_array[:,(col_size):(col_size+1)].flatten() #molar percentages of relevant formulations
all_size_training = allforms[:,(col_size):(col_size+1)].flatten() #molar percentages of all formulations


#create models and make fits, 'liblinear' used due to small dataset sizes
focused_model = linear_model.LogisticRegression(solver='liblinear')
all_model = linear_model.LogisticRegression(solver='liblinear')

focus = focused_model.fit(focus_percentage_training, focus_size_training)
broad = all_model.fit(all_percentage_training, all_size_training)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
